In [67]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt

In [68]:
df_zillow = pd.read_csv("df_zillow.csv")

In [69]:
lat_range = max(df_zillow["latitude"]) - min(df_zillow["latitude"])
lon_range = max(df_zillow["longitude"]) - min(df_zillow["longitude"])

a_lat = min(df_zillow["latitude"])
a_lon = min(df_zillow["longitude"])

lat_scale = []
lon_scale = []

for i in range(63):
    lat_scale.append((df_zillow["latitude"][i] - a_lat) / lat_range)
    lon_scale.append((df_zillow["longitude"][i] - a_lon) / lon_range)

df_zillow["latScale"] = lat_scale
df_zillow["lonScale"] = lon_scale

In [101]:
Y = df_zillow["price"]
Y_scale = (Y - min(Y)) / (max(Y) - min(Y))
Y_bins = (np.floor(Y_scale*10))
for i in range(len(Y_bins)):
    if Y_bins[i] == 10.0:
        Y_bins[i] = Y_bins[i] - 1 
df_zillow["Y"] = Y_bins
print(df_zillow)

df_zillow_train, df_zillow_test = train_test_split(df_zillow, test_size=0.2)
Y_train = df_zillow_train["Y"]
X_train = df_zillow_train[["latScale", "lonScale", "zestimate", "bedrooms", "bathrooms"]]
Y_test = df_zillow_test["Y"]
X_test = df_zillow_test[["latScale", "lonScale", "zestimate", "bedrooms", "bathrooms"]]

df_zillow_full = df_zillow[["latScale", "lonScale", "zestimate", "bedrooms", "bathrooms", "Y"]]
print(df_zillow_full)

    dateSold   propertyType                                          address  \
0   Not Sold  SINGLE_FAMILY                159 E 71st St, New York, NY 10021   
1   Not Sold  SINGLE_FAMILY                132 E 62nd St, New York, NY 10065   
2   Not Sold  SINGLE_FAMILY                 10 E 64th St, New York, NY 10065   
3   Not Sold  SINGLE_FAMILY                161 E 63rd St, New York, NY 10065   
4   Not Sold  SINGLE_FAMILY                248 W 71st St, New York, NY 10023   
..       ...            ...                                              ...   
58  Not Sold  SINGLE_FAMILY              17644 Kildare Rd, Jamaica, NY 11432   
59  Not Sold  SINGLE_FAMILY   99-24 Russell Street, Hamilton Beach, NY 11414   
60  Not Sold  SINGLE_FAMILY  116-17 126th Street, South Ozone Park, NY 11420   
61  Not Sold  SINGLE_FAMILY    110-42 207th Street, Queens Village, NY 11429   
62  Not Sold  SINGLE_FAMILY                5027 245th St, Flushing, NY 11362   

    priceChange  zestimate     price  b

In [89]:
model = Sequential()
model.add(Dense(units=50, activation = 'relu', input_dim = 5))
model.add(Dense(units=100, activation = 'relu'))
model.add(Dense(units=20, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units=10, activation = 'softmax'))

In [90]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = 'accuracy')

In [91]:
model.fit(X_train, Y_train, epochs = 500, batch_size = 50)

Epoch 1/500
1/1 [==============================] - 0s 316ms/step - loss: 685586.6250 - accuracy: 0.0400
Epoch 2/500
1/1 [==============================] - 0s 4ms/step - loss: 688129.6250 - accuracy: 0.0600
Epoch 3/500
1/1 [==============================] - 0s 4ms/step - loss: 394829.4688 - accuracy: 0.1200
Epoch 4/500
1/1 [==============================] - 0s 3ms/step - loss: 347361.4688 - accuracy: 0.1400
Epoch 5/500
1/1 [==============================] - 0s 3ms/step - loss: 332155.8438 - accuracy: 0.0600
Epoch 6/500
1/1 [==============================] - 0s 3ms/step - loss: 315082.4062 - accuracy: 0.0400
Epoch 7/500
1/1 [==============================] - 0s 3ms/step - loss: 249784.9531 - accuracy: 0.0400
Epoch 8/500
1/1 [==============================] - 0s 3ms/step - loss: 212794.0781 - accuracy: 0.0600
Epoch 9/500
1/1 [==============================] - 0s 4ms/step - loss: 199575.2031 - accuracy: 0.0800
Epoch 10/500
1/1 [==============================] - 0s 3ms/step - loss: 156231.7

In [96]:
predictions = model.predict(X_test)
pred_class = []
for i in range(len(X_test)):
    pred_class.append(Y_bins[np.argmax(predictions)])
print(pred_class)


1/1 [==============================] - 0s 19ms/step
[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]


In [98]:
matrix = metrics.confusion_matrix(Y_test, pred_class)
print(matrix)

[[0 0 0 6 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 2 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0]]
